In [56]:
import pandas as pd
import math
from tqdm import tqdm


# CS Rankings Author info

Taken from https://github.com/emeryberger/CSrankings/tree/gh-pages which is the repo behind https://csrankings.org 

Run `make generated-author-info.csv` which should make the csv. Each row contains the score for every author, year, and conference combination. This means that there are multiple rows for each professor.

# CS Professors

Taken from https://drafty.cs.brown.edu/csprofessors but they hide the data. So, after reading source code, exploit a [thing they left in](https://github.com/brownhci/drafty/blob/212bd995c857a34c74c7a71d67e1556c1ca7ea97/backend/src/controllers/datasharing.ts#L31) during development and use https://drafty.cs.brown.edu/data/csv/csprofessors/csprofessors_93318b344889ccef41d46b5f83d63de5 

# Placement Rank

Taken from https://drafty.cs.brown.edu/csopenrankings/placement-rank.html which is just copy and paste (and `M-x query-replace <tab> ,`). I think I could have done this myself, but why do that when someone else has already done the work. 

# Best Paper awards

A collection of best paper awards are listed on https://jeffhuang.com/best_paper_awards/

However, I needed to do some html parsing in order to get the data into a csv format. That is done in [another notebook](./best-paper.ipynb)


# [Data](https://store.sachiniyer.com/s/9NczHAmPJFiQgys)


In [4]:
dfcsrankings = pd.read_csv("data/generated-author-info.csv")
dfcsrankings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199906 entries, 0 to 199905
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   name           199906 non-null  object 
 1   dept           199906 non-null  object 
 2   area           199906 non-null  object 
 3   count          199906 non-null  float64
 4   adjustedcount  199906 non-null  float64
 5   year           199906 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 9.2+ MB


In [123]:
dfcsprofs = pd.read_csv("data/csprofessors.csv")
dfcsprofs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   UniqueId    5630 non-null   int64  
 1   FullName    5629 non-null   object 
 2   University  5630 non-null   object 
 3   JoinYear    5512 non-null   float64
 4   SubField    5625 non-null   object 
 5   Bachelors   5331 non-null   object 
 6   Doctorate   5618 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 308.0+ KB


In [16]:
dfplacement = pd.read_csv("data/placement-rank.csv", delimiter='|')
dfplacement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   num             183 non-null    int64  
 1   university      183 non-null    object 
 2   size            115 non-null    float64
 3   bachelors       168 non-null    float64
 4   doctorate       159 non-null    float64
 5   placementscore  183 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 8.7+ KB


In [11]:
dfbestpaper = pd.read_csv("data/best-paper.csv", delimiter='|')
dfbestpaper = dfbestpaper.applymap(lambda x: x.strip() if isinstance(x, str) else x)
dfbestpaper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Author      4815 non-null   object
 1   University  4815 non-null   object
 2   Conference  4815 non-null   object
 3   Area        4815 non-null   object
 4   Year        4815 non-null   int64 
 5   Coauthors   4815 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 225.8+ KB


In [13]:
dfpreference = pd.read_csv("data/preferences.csv")
dfpreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   conference  99 non-null     object
 1   preference  99 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [104]:
university_alias = {}
with open('data/universities.csv', 'r') as file:
    for line in file:
        unis = line.split(',')
        unis.pop()
        first = unis.pop(0)
        university_alias[first] = unis 
count = 5
for k, v in university_alias.items():
    if count < 0:
        break
    count -= 1
    print(f"{k} {v}")
print(len(university_alias))

University of Kentucky []
Colorado State University []
University of Illinois at Chicago []
Duke University []
Florida International University []
UCCS []
186


In [106]:
def get_alias(uni):
    if uni in university_alias:
        return uni
    for k, v in university_alias.items():
        if uni in v:
            return k
    return None
    

# Methodology

I want to do a little better than following the rankings blindly, here are the factors I want to consider.

## Data

1. Professor Conference Score (essentially the csrankings.org score)
2. Best Paper Awards
3. Placement Rank (taken from the drafty placement rank, which is pagerank on professors original universities and then employment universities)
4. Variance of Professors

## Method

### Professor Score
A professor will be given a score based on their conference score and best papers. 

Both will have weights by:
- Will be weighted by logarithmically by the year published (the older, the less score)
- Will be weighted by how interested I am in the area (weights defined manually)

### Meta Scores

#### Placement Rank
A Placement Rank will be computed based on the professor score (the places where more important professors originate from matters more).

Logarithmic scale will be used to normalize the professor scores here.

#### Professor Score Variance
Variance of professor score will then be computed

#### Professor Score Sum
A sum of professors score will then be computed

### Final Score

I will take the Meta Scores, normalize them with min max scaling. Then I will apply some weights and get my results.



# Computing Professor Scores

- I will use the `generated-author-info` dataset for each professors to give their scores for each unique year-conference combination
- I will use the `best-paper` dataset for each professor to give a bonus for each professor with a best paper
- I will use `preference` dataset (which I manually created) to give weight preferences.


In [63]:
def year_weight(year):
    # 1970 - 2023
    year = 2023 - year + 1
    year *= 0.5
    return math.log(year)


In [64]:
def area_weight(area):
    # score of 1-5
    area_score = dfpreference[dfpreference['conference'] == area]['preference'].values[0]
    return math.log(area_score)

In [65]:
dfauthorscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfcsrankings.iterrows(), desc="Prof Scores", unit="item", total=len(dfcsrankings)):
    name = row['name']
    university = row['dept']

    year = row['year']
    area = row['area']

    yearweight = year_weight(year)
    areaweight = area_weight(area)


    count = row['adjustedcount'] * yearweight * areaweight

    existing_rows = dfauthorscore[dfauthorscore['name'] == name]
    if existing_rows.empty:
        dfauthorscore.loc[len(dfauthorscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfauthorscore.at[idx, 'score'] += count


    

Prof Scores:   0%|                                                                                                                                                                                                           | 0/199906 [00:00<?, ?item/s]

Prof Scores:   0%|                                                                                                                                                                                                 | 67/199906 [00:00<04:58, 669.55item/s]

Prof Scores:   0%|▏                                                                                                                                                                                               | 165/199906 [00:00<03:54, 851.28item/s]

Prof Scores:   0%|▎                                                                                                                                                                                              | 292/199906 [00:00<03:12, 1039.64item/s]

Prof Scores:   0%|▍                                                                                                                                                                                              | 422/199906 [00:00<02:55, 1139.08item/s]

...

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 199708/199906 [04:58<00:00, 482.95item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 199757/199906 [04:58<00:00, 481.57item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 199806/199906 [04:58<00:00, 469.62item/s]

Prof Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 199858/199906 [04:58<00:00, 482.02item/s]

Prof Scores: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 199906/199906 [04:58<00:00, 669.02item/s]

In [75]:
dfpaperscore = pd.DataFrame(columns=['name', 'university', 'score'])

for index, row in tqdm(dfbestpaper.iterrows(), desc="Best Paper Scores", unit="item", total=len(dfbestpaper)):
    name = row['Author']
    university = row['University']

    year = row['Year']
    area = row['Area']
    coauthors = row['Coauthors']

    yearweight = year_weight(year)
    areaweight = area_weight(area)

    count = 10 * yearweight * areaweight

    existing_rows = dfpaperscore[dfpaperscore['name'] == name]
    if existing_rows.empty:
        dfpaperscore.loc[len(dfpaperscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfpaperscore.at[idx, 'score'] += count
       




Best Paper Scores:   0%|                                                                                                                                                                                                       | 0/4815 [00:00<?, ?item/s]

Best Paper Scores:   1%|█▉                                                                                                                                                                                           | 50/4815 [00:00<00:09, 492.02item/s]

Best Paper Scores:   2%|███▉                                                                                                                                                                                        | 101/4815 [00:00<00:09, 499.88item/s]

Best Paper Scores:   3%|██████                                                                                                                                                                                      | 156/4815 [00:00<00:08, 522.21item/s]

Best Paper Scores:   4%|████████▏                                                                                                                                                                                   | 209/4815 [00:00<00:08, 517.84item/s]

...

Best Paper Scores:  96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌       | 4624/4815 [00:09<00:00, 494.00item/s]

Best Paper Scores:  97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋     | 4677/4815 [00:09<00:00, 503.01item/s]

Best Paper Scores:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌   | 4728/4815 [00:09<00:00, 493.09item/s]

Best Paper Scores:  99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 4778/4815 [00:09<00:00, 486.69item/s]

Best Paper Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4815/4815 [00:09<00:00, 507.25item/s]

In [82]:
dfauthorscore.sample(5)

,name,university,score
6072,Jan Hoffmann 0002,Carnegie Mellon University,3.790165
1403,Barbara J. Grosz,Harvard University,19.244281
9234,Mehmet Can Vuran,University of Nebraska,0.737858
3979,Ettore Merlo,Polytechnique Montreal,2.185226
6526,Jie Tang 0001,Tsinghua University,30.309512


In [81]:
dfpaperscore.sample(5)

,name,university,score
773,Mao Yang,Microsoft,4.804530
3206,Jingwen Ou,University of British Columbia,13.488021
25,Shanghang Zhang,University of California Berkeley,6.525709
2290,Stephany Bellomo,Carnegie Mellon University,10.425470
424,Kayvan Memarian,University Of Cambridge,12.419530


In [116]:
dfscore = dfauthorscore.copy()
for index, row in tqdm(dfpaperscore.iterrows(), desc="Merge Scores", unit="item", total=len(dfpaperscore)):
    name = row['name']
    university = row['university']
    count = row['score']

    existing_rows = dfscore[dfscore['name'] == name]

    if existing_rows.empty:
        dfscore.loc[len(dfscore)] = [name, university, count]
    else:
        for idx in existing_rows.index:
            dfscore.at[idx, 'score'] += count


Merge Scores:   0%|                                                                                                                                                                                                            | 0/4099 [00:00<?, ?item/s]

Merge Scores:   1%|█▌                                                                                                                                                                                                | 32/4099 [00:00<00:12, 314.81item/s]

Merge Scores:   2%|███                                                                                                                                                                                               | 64/4099 [00:00<00:13, 308.84item/s]

Merge Scores:   2%|████▍                                                                                                                                                                                             | 95/4099 [00:00<00:13, 302.58item/s]

Merge Scores:   3%|█████▉                                                                                                                                                                                           | 126/4099 [00:00<00:13, 300.65item/s]

...

Merge Scores:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋    | 4008/4099 [00:13<00:00, 286.12item/s]

Merge Scores:  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████   | 4037/4099 [00:13<00:00, 284.91item/s]

Merge Scores:  99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍ | 4066/4099 [00:13<00:00, 284.62item/s]

Merge Scores: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 4095/4099 [00:13<00:00, 281.46item/s]

Merge Scores: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4099/4099 [00:13<00:00, 304.82item/s]

In [117]:
dfscore['university'] = dfscore['university'].apply(get_alias)
dfscore = dfscore.dropna(subset=['university'])

In [118]:
dfscore.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7493 entries, 4 to 19418
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        7493 non-null   object 
 1   university  7493 non-null   object 
 2   score       7493 non-null   float64
dtypes: float64(1), object(2)
memory usage: 234.2+ KB


In [119]:
dfscore.sample(5)

,name,university,score
7156,Jun Wang 0001,University of Central Florida,3.409191
18147,Yin Tat Lee & Aaron Sidford,Massachusetts Institute of Technology,25.902904
5146,Hany H. Ammar,West Virginia University,1.092494
18926,John P. John,University of Washington,28.827181
14228,Vassilis J. Tsotras,Univ. of California - Riverside,18.943815


# Computing Professor Placement Scores

I will compute three scores, one for relationship between bachelor with placement, bachelor with phd, and phd with placement.

I will do this by mapping dfcsprofs onto dfscore, which will let me value each one of the connections.


In [130]:
dfscoreplacement = dfscore.copy()
dfscoreplacement['name'] = dfscoreplacement['name'].str.lower()

dfcsprofsplacement = dfcsprofs.copy()
dfcsprofsplacement['name'] = dfcsprofsplacement['FullName'].str.lower()
dfcsprofsplacement['university'] = dfcsprofsplacement['University'].apply(get_alias)
dfcsprofsplacement['bachelors'] = dfcsprofsplacement['Bachelors'].apply(get_alias)
dfcsprofsplacement['doctorate'] = dfcsprofsplacement['Doctorate'].apply(get_alias)
dfcsprofsplacement.dropna(subset=['university'])
dfcsprofsplacement.dropna(subset=['bachelors'])
dfcsprofsplacement.dropna(subset=['doctorate'])

mapping_dict = dict(zip(dfcsprofsplacement['name'], zip(dfcsprofsplacement['university'],
                                                        dfcsprofsplacement['bachelors'],
                                                        dfcsprofsplacement['doctorate'])))
dfscoreplacement[['placement', 'bachelors', 'doctorate']] = dfscoreplacement['name'].map(mapping_dict).apply(pd.Series)

dfscoreplacement = dfscoreplacement.dropna(subset=['placement', 'bachelors', 'doctorate'])

dfscoreplacement.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 864 entries, 30 to 19109
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        864 non-null    object 
 1   university  864 non-null    object 
 2   score       864 non-null    float64
 3   placement   864 non-null    object 
 4   bachelors   864 non-null    object 
 5   doctorate   864 non-null    object 
dtypes: float64(1), object(5)
memory usage: 47.2+ KB


In [132]:
dfscoreplacement.sample(5)

,name,university,score,placement,bachelors,doctorate
13175,stephen g. kobourov,University of Arizona,8.182133,University of Arizona,Dartmouth College,Johns Hopkins University
5857,j. alex halderman,University of Michigan,11.486125,University of Michigan,Princeton University,Princeton University
10635,paul a. fishwick,University of Texas at Dallas,0.000000,University of Texas at Dallas,Pennsylvania State University,University of Pennsylvania
3099,david i. w. levin,University of Toronto,1.741672,University of Toronto,University of Toronto,University of British Columbia
7970,lawrence carin,Duke University,56.999089,Arizona State University,University of Maryland - College Park,University of Maryland - College Park
